In [1]:
!wget -p https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip

--2021-06-09 19:16:34--  https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip
Resolving www.anatel.gov.br (www.anatel.gov.br)... 200.0.81.81, 2801:80:c90:c1da:da0:2312:2000:1331
Connecting to www.anatel.gov.br (www.anatel.gov.br)|200.0.81.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 258274514 (246M) [application/x-zip-compressed]
Saving to: ‘www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip’

www.anatel.gov.br/d   8%[>                   ]  20.25M   938KB/s    in 23s     

2021-06-09 19:17:00 (904 KB/s) - Read error at byte 21233664/258274514 (Connection reset by peer). Retrying.

--2021-06-09 19:17:01--  (try: 2)  https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip
Connecting to www.anatel.gov.br (www.anatel.gov.br)|200.0.81.81|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 258274514 (246M), 237040850

In [2]:
import pandas as pd
import numpy as np
import os
from zipfile import ZipFile

In [3]:
pasta = '/content/www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos'
broadband = os.path.join(pasta, 'acessos_banda_larga_fixa.zip')

In [4]:
#decodificando os arquivos
  #Anatel separa os arquivos por biênio
  #Entre 2007 e 2010 os dados eram fornecidos por trimestre, a partir de 2011 os dados passam a ser mensais
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2007-2010.csv') as f:
    b_1 = pd.read_csv(f,sep=';', encoding='utf-8')

In [5]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2011-2012.csv') as f:
    b_2 = pd.read_csv(f,sep=';', encoding='utf-8')

In [6]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2013-2014.csv') as f:
    b_3 = pd.read_csv(f,sep=';', encoding='utf-8')

In [7]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2015-2016.csv') as f:
    b_4 = pd.read_csv(f,sep=';', encoding='utf-8')

In [8]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2017-2018.csv') as f:
    b_5 = pd.read_csv(f,sep=';', encoding='utf-8')

In [9]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2019-2020.csv') as f:
    b_6 = pd.read_csv(f,sep=';', encoding='utf-8')

In [10]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2021.csv') as f:
    b_7 = pd.read_csv(f,sep=';', encoding='utf-8')

In [11]:
#junção das bases
bl = b_1.append([b_2, b_3, b_4, b_5, b_6, b_7], ignore_index=True)

In [12]:
#renomeando variaveis
bl.rename(columns={'Ano': 'ano','Mês':'mes', 'Grupo Econômico':'grupo_economico', 'Empresa':'empresa',
                 'CNPJ':'cnpj', 'Porte da Prestadora':'porte_empresa', 'UF':'sigla_uf', 'Município':'municipio',
                 'Código IBGE Município':'id_municipio', 'Faixa de Velocidade':'velocidade', 'Tecnologia':'tecnologia',
                 'Meio de Acesso':'transmissao', 'Acessos':'acessos', 'Tipo de Pessoa': 'pessoa'}, inplace=True)

In [13]:
#organização das variáveis
bl.drop(['grupo_economico', 'municipio', 'pessoa'], axis=1, inplace=True)
bl = bl[['ano', 'mes', 'sigla_uf', 'id_municipio', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'transmissao', 'velocidade', 'acessos']]

In [27]:
bl.sort_values(['ano', 'mes', 'id_municipio'], inplace=True)

In [28]:
#salvar arquivos no Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
#exportação para csv
bl.to_csv('/content/drive/MyDrive/br_anatel/banda_larga_fixa/output/banda_larga_microdados', index=False)

In [34]:
#drop das variáveis ligada a empresas
bl_mun = bl.drop(['empresa','cnpj', 'porte_empresa'], axis = 1)
#collapse para acessos
bl_mun['acessos_total'] = bl_mun.groupby(['ano', 'mes', 'id_municipio', 'tecnologia', 'transmissao', 'velocidade'])['acessos'].transform(np.sum)
#ordenamento das variaveis
bl_mun.sort_values(['ano', 'mes', 'id_municipio', 'tecnologia', 'transmissao', 'velocidade'], inplace=True)
#exclusão das variaveis duplicadas
bl_mun.drop_duplicates(subset=['ano', 'mes', 'id_municipio', 'tecnologia', 'transmissao', 'velocidade'], keep ='first', inplace = True)
#exclusão da coluna de acessos
bl_mun.drop('acessos', axis=1, inplace=True)
#renome da coluna de acessos total
bl_mun.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [37]:
bl_mun.to_csv('/content/drive/MyDrive/br_anatel/banda_larga_fixa/output/banda_larga_municipios', index=False)